In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.python.keras import applications
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from tensorflow.python.keras.optimizers import RMSprop
import numpy as np
import pandas as pd

In [20]:
#parameters
classes = 12
batch_size=128
train_total = 3783
validation_total = 967

In [3]:
model_resnet = ResNet50(include_top=False, weights = 'imagenet',input_shape=(224,224,3))

In [6]:
x = model_resnet.get_layer('avg_pool').output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [7]:
model = Model(inputs=model_resnet.input, outputs=x)

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 112, 112, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [10]:
for layer in model_resnet.layers:
    layer.trainable= False

In [11]:
model.get_layer('res5a_branch2a').trainable= True
model.get_layer('bn5a_branch2a').trainable= True
model.get_layer('res5a_branch2b').trainable= True
model.get_layer('bn5a_branch2b').trainable= True
model.get_layer('res5a_branch2c').trainable= True
model.get_layer('res5a_branch1').trainable= True
model.get_layer('bn5a_branch2c').trainable= True
model.get_layer('bn5a_branch1').trainable= True
model.get_layer('res5b_branch2a').trainable= True
model.get_layer('bn5b_branch2a').trainable= True
model.get_layer('res5b_branch2b').trainable= True
model.get_layer('bn5b_branch2b').trainable= True
model.get_layer('res5b_branch2c').trainable= True
model.get_layer('bn5b_branch2c').trainable= True
model.get_layer('res5c_branch2a').trainable= True
model.get_layer('bn5c_branch2a').trainable= True
model.get_layer('res5c_branch2b').trainable= True
model.get_layer('bn5c_branch2b').trainable= True
model.get_layer('res5c_branch2c').trainable= True
model.get_layer('bn5c_branch2c').trainable= True

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [21]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 3820 images belonging to 12 classes.
Found 930 images belonging to 12 classes.


In [26]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-5),
             metrics=['accuracy'])

In [29]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=50,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/50
95s - loss: 0.2417 - acc: 0.9191 - val_loss: 0.1696 - val_acc: 0.9376
Epoch 2/50
82s - loss: 0.2254 - acc: 0.9249 - val_loss: 0.1613 - val_acc: 0.9355
Epoch 3/50
86s - loss: 0.2250 - acc: 0.9218 - val_loss: 0.1491 - val_acc: 0.9462
Epoch 4/50
80s - loss: 0.2078 - acc: 0.9273 - val_loss: 0.1755 - val_acc: 0.9312
Epoch 5/50
79s - loss: 0.2058 - acc: 0.9284 - val_loss: 0.1453 - val_acc: 0.9484
Epoch 6/50
82s - loss: 0.2078 - acc: 0.9262 - val_loss: 0.1471 - val_acc: 0.9441
Epoch 7/50
79s - loss: 0.2041 - acc: 0.9249 - val_loss: 0.1585 - val_acc: 0.9312
Epoch 8/50
82s - loss: 0.2018 - acc: 0.9275 - val_loss: 0.1631 - val_acc: 0.9366
Epoch 9/50
78s - loss: 0.1905 - acc: 0.9304 - val_loss: 0.1672 - val_acc: 0.9323
Epoch 10/50
77s - loss: 0.1864 - acc: 0.9356 - val_loss: 0.1539 - val_acc: 0.9441
Epoch 11/50
80s - loss: 0.1964 - acc: 0.9333 - val_loss: 0.1414 - val_acc: 0.9419
Epoch 12/50
78s - loss: 0.1826 - acc: 0.9336 - val_loss: 0.1545 - val_acc: 0.9409
Epoch 13/50
79s - loss: 0

In [28]:
model.save_weights('resnet50_bottom_fixed.hdf5')

In [ ]:
model.load_weights('resnet50_bottom_fixed.hdf5')